# Imports

In [19]:
import numpy as np
import pandas as pd
import json
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

import tensorflow as tf
import keras
import optuna

# Type hinting for better code documentation
from typing import Tuple, Union, Dict, Any

## Auxiliary Functions

In [20]:
def create_sequences(data: np.ndarray, timesteps: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Create sequences of data for time series forecasting.

    This function takes a time series dataset and splits it into sequences of a specified length (timesteps).
    Each sequence is used as input (X), and the value immediately preceding the sequence is used as the target (y).
    
    Note: This creates a look-ahead prediction approach - we use n timesteps to predict the value
    that came right before those timesteps. This is common in certain types of time series models.

    Parameters:
    - data (numpy.ndarray): The time series data to be split into sequences.
    - timesteps (int): The number of time steps in each sequence.

    Returns:
    - X (numpy.ndarray): Array of input sequences, where each sequence has a shape of (timesteps, features).
    - y (numpy.ndarray): Array of target values corresponding to each sequence.
    """
    X, y = [], []
    # Calculate how many complete sequences can be extracted from the data
    num_samples = len(data) // timesteps  # Ensure only full sequences are used

    # Iterate backwards through the dataset to create sequences
    for i in range(num_samples):
        # Calculate start and end indices for this sequence
        start = len(data) - (i + 1) * timesteps
        end = start + timesteps

        if start >= 0:  # Ensure valid indexing (there's at least one value before the sequence)
            X.append(data[start:end])           # Sequence becomes input feature
            y.append(data[start - 1])           # Value before sequence becomes target

    return np.array(X), np.array(y)


def generate_model_checkpoint(model_weights_location: str) -> keras.callbacks.ModelCheckpoint:
    """
    Generate a ModelCheckpoint callback for saving the best model weights.
    
    ModelCheckpoint is a crucial callback that prevents overfitting by saving weights
    only when the model improves on the validation set.

    This function creates a Keras ModelCheckpoint callback that monitors the validation loss
    during training and saves the model weights whenever the validation loss improves.

    Parameters:
    - model_weights_location (str): The file path where the model weights will be saved.

    Returns:
    - keras.callbacks.ModelCheckpoint: A ModelCheckpoint callback configured to save the best weights.
    """
    return keras.callbacks.ModelCheckpoint(
        filepath=model_weights_location,
        monitor='val_loss',            # Monitor validation loss
        save_best_only=True,           # Only save when val_loss improves
        save_weights_only=True,        # Save only weights, not entire model
        mode='min'                     # Lower validation loss is better
    )


def generate_early_stop() -> keras.callbacks.EarlyStopping:
    """
    Generate an EarlyStopping callback for training.
    
    Early stopping prevents overfitting by halting training when performance
    on validation data stops improving.

    This function creates a Keras EarlyStopping callback that monitors the validation loss
    during training and stops the training process if the validation loss does not improve
    for a specified number of epochs (patience). It also restores the model weights to the
    best weights observed during training.

    Returns:
    - keras.callbacks.EarlyStopping: An EarlyStopping callback configured to monitor validation loss.
    """
    return keras.callbacks.EarlyStopping(
        monitor='val_loss',           # Monitor validation loss
        patience=5,                   # Wait for 5 epochs of no improvement before stopping
        restore_best_weights=True     # Restore to the best weights found during training
    )

# Constants

In [21]:
# Control flags for different phases of the workflow
IS_TO_FINE_TUNE = False
IS_TO_BENCHMARK = False

# Model configuration parameters
SEQ_LENGTH = 1
SEED = 64

# Model weight initialization strategies (for reproducibility)
KERNEL_INITIALIZER = keras.initializers.GlorotUniform(seed=SEED)
RECURRENT_INITIALIZER = keras.initializers.Orthogonal(seed=SEED)

# File paths
PATH = "../dataset"
DATA_FILENAME = f"{PATH}/data.csv"

# Checkpoint callbacks to save the best model weights during training
RENEWABLES_CHECKPOINT = generate_model_checkpoint(f"{PATH}/best_renewables_model.weights.h5")
NON_RENEWABLES_CHECKPOINT = generate_model_checkpoint(f"{PATH}/best_non_renewables_model.weights.h5")

# Data Loading

In [22]:
# Load the dataset from CSV file
df = pd.read_csv(DATA_FILENAME)
df.fillna(0, inplace=True)

# Convert the timestamp column to datetime format and set it as the index
df['Data e Hora'] = pd.to_datetime(df['Data e Hora'])
df = df.set_index('Data e Hora')
df.head()

,producao_total,producao_renovavel,hidrica,eolica,solar,ondas,biomassa,producao_nao_renovavel,carvao,gas_natural,termica,producao_por_bombagem,importacao,exportacao
Data e Hora,,,,,,,,,,,,,,
2011-01-01,107.975628,73.076341,63.952042,2.142578,0.122491,0.0,6.859230,34.484478,4.3443,28.367837,1.772341,0.414810,12.4186,0.0852
2011-01-02,109.657562,71.219553,62.732572,1.469945,0.197608,0.0,6.819428,37.876984,0.7383,35.343969,1.794715,0.561025,20.8615,0.0000
2011-01-03,150.642916,74.660080,63.158904,4.503582,0.166772,0.0,6.830821,74.808327,14.9438,55.179956,4.684572,1.174509,13.4903,1.0916
2011-01-04,159.833303,101.106584,64.904843,28.734425,0.256511,0.0,7.210805,58.567678,4.8496,48.985982,4.732096,0.159041,9.3552,0.7629
2011-01-05,170.854279,133.687873,67.566754,58.869586,0.208719,0.0,7.042814,36.428947,4.6337,27.003927,4.791320,0.737460,6.7881,6.8363


## Train, Validation, and Test split

In [23]:
# Extract the target columns for renewable and non-renewable energy production
renewables_df = df["producao_renovavel"]
non_renewables_df = df["producao_nao_renovavel"]

# Define the split ratios
train_ratio = 0.8
val_ratio = 0.2
test_ratio = 0.2

# Calculate the split points in terms of number of samples
train_split = int(df.shape[0] * train_ratio * (1 - val_ratio))
val_split = int(df.shape[0] * train_ratio * val_ratio)
test_split = int(df.shape[0] * test_ratio)

# Split the renewable energy production dataset
renewables_df_train = renewables_df[:train_split]
renewables_df_val = renewables_df[train_split:train_split + val_split]
renewables_df_test = renewables_df[:test_split]

# Split the non-renewable energy production dataset
non_renewables_df_train = non_renewables_df[:train_split]
non_renewables_df_val = non_renewables_df[train_split:train_split + val_split]
non_renewables_df_test = non_renewables_df[:test_split]

## Preprocess Dataset for Model Training

In [24]:
# Scaling is important for neural networks to ensure stable and fast convergence
renewables_scaler = MinMaxScaler(feature_range=(0, 1))
non_renewables_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the renewable energy datasets
renewables_df_train_scaled = renewables_scaler.fit_transform(renewables_df_train.values.reshape(-1, 1))
renewables_df_val_scaled = renewables_scaler.transform(renewables_df_val.values.reshape(-1, 1))
renewables_df_test_scaled = renewables_scaler.transform(renewables_df_test.values.reshape(-1, 1))

# Scale the non-renewable energy datasets
non_renewables_df_train_scaled = non_renewables_scaler.fit_transform(non_renewables_df_train.values.reshape(-1, 1))
non_renewables_df_val_scaled = non_renewables_scaler.transform(non_renewables_df_val.values.reshape(-1, 1))
non_renewables_df_test_scaled = non_renewables_scaler.transform(non_renewables_df_test.values.reshape(-1, 1))

# Create sequence datasets for time series modeling using the function defined earlier
# For renewable energy data:
renewables_X_train, renewables_y_train = create_sequences(renewables_df_train_scaled, SEQ_LENGTH)
renewables_X_val, renewables_y_val = create_sequences(renewables_df_val_scaled, SEQ_LENGTH)
renewables_X_test, renewables_y_test = create_sequences(renewables_df_test_scaled, SEQ_LENGTH)

# For non-renewable energy data:
non_renewables_X_train, non_renewables_y_train = create_sequences(non_renewables_df_train_scaled, SEQ_LENGTH)
non_renewables_X_val, non_renewables_y_val = create_sequences(non_renewables_df_val_scaled, SEQ_LENGTH)
non_renewables_X_test, non_renewables_y_test = create_sequences(non_renewables_df_test_scaled, SEQ_LENGTH)

# Print dataset sizes to verify the splits
print("Renewable energy production time series split:")
print("Training set size:", renewables_X_train.shape)
print("Validation set size:", renewables_X_val.shape)
print("Test set size:", renewables_X_test.shape)

print()
print("Non-renewable energy production time series split:")
print("Training set size:", non_renewables_X_train.shape)
print("Validation set size:", non_renewables_X_val.shape)
print("Test set size:", non_renewables_X_test.shape)

Renewable energy production time series split:
Training set size: (3354, 1, 1)
Validation set size: (838, 1, 1)
Test set size: (1048, 1, 1)

Non-renewable energy production time series split:
Training set size: (3354, 1, 1)
Validation set size: (838, 1, 1)
Test set size: (1048, 1, 1)


# Deep Learning Models

## Selected Architecture

In [25]:
def regression_model_builder(
        input_shape: Tuple[int, int, int],
        first_lstm_layer_size: int,
        second_lstm_layer_size: int,
        dense_layer_size: int,
        dense_layer_activation_func: str = "relu",
        kernel_initializer: Union[str, keras.initializers.GlorotUniform] = "glorot_uniform",
        recurrent_initializer: Union[str, keras.initializers.Orthogonal] = "orthogonal"
    ) -> keras.Model:
    """
    Build a stacked LSTM model for time series regression.
    
    Architecture:
    1. Input layer
    2. First LSTM layer with return_sequences=True to pass all outputs to next LSTM
    3. Second LSTM layer that processes outputs from first LSTM
    4. Dense layer with activation function (typically ReLU)
    5. Output layer (single neuron for regression)
    
    Parameters:
    - input_shape: Shape of input data (samples, timesteps, features)
    - first_lstm_layer_size: Number of units in first LSTM layer
    - second_lstm_layer_size: Number of units in second LSTM layer
    - dense_layer_size: Number of units in dense layer
    - dense_layer_activation_func: Activation function for dense layer
    - kernel_initializer: Weight initialization for kernels
    - recurrent_initializer: Weight initialization for recurrent connections
    
    Returns:
    - Compiled Keras model ready for training
    """
    return keras.Sequential([
        # Input layer defines the shape of input data
        keras.layers.Input(shape=input_shape),
        
        # First LSTM layer processes the input sequence and outputs a sequence
        keras.layers.LSTM(first_lstm_layer_size, return_sequences=True, 
                          kernel_initializer=kernel_initializer, 
                          recurrent_initializer=recurrent_initializer, 
                          name="first_recurrent_layer"),
        
        # Second LSTM layer processes the sequence from first LSTM and outputs a single vector
        keras.layers.LSTM(second_lstm_layer_size, 
                          kernel_initializer=kernel_initializer, 
                          recurrent_initializer=recurrent_initializer, 
                          name="second_recurrent_layer"),
        
        # Dense layer for further processing and introducing non-linearity
        keras.layers.Dense(dense_layer_size, 
                          activation=dense_layer_activation_func, 
                          kernel_initializer=kernel_initializer, 
                          name="dense_layer"),
        
        # Output layer - single neuron for regression with no activation (linear)
        keras.layers.Dense(1, kernel_initializer=kernel_initializer, name="output_layer")
    ])

## Model fine-tuning

Next is implemented a fine-tuning process for the LSTM-based model designed to forecast renewable and non-renewable energy production. The fine-tuning is conducted using Optuna, a state-of-the-art hyperparameter optimization framework. The workflow includes the following steps:
1. **Objective Function Definition**: An objective function is defined to build, train, and evaluate models based on hyperparameters suggested by Optuna.
2. **Hyperparameter Optimization**: Multiple trials are executed to identify the optimal combination of hyperparameters, such as LSTM layer sizes, dense layer size, and learning rate.
3. **Model Training**: A final model is trained using the best hyperparameters, and the best weights are saved for future use.
4. **Performance Evaluation**: Predictions are made on the test set, and the model's performance is assessed using metrics such as Mean Absolute Percentage Error (MAPE) and R-squared (R²).
5. **Hyperparameter Persistence**: The best hyperparameters are saved to a JSON file for reproducibility and future reference.

If fine-tuning is disabled (`IS_TO_FINE_TUNE = False`), the cell instead loads previously saved hyperparameters from a JSON file to avoid redundant optimization.

### Renewables

In [ ]:
if IS_TO_FINE_TUNE:
    print(f"Fine tunning 'renewable energy production' time series")

    def objective(trial: optuna.trial.Trial):
        """
        Objective function for Optuna hyperparameter optimization.
        
        This function:
        1. Suggests hyperparameter values to try
        2. Builds a model with those hyperparameters
        3. Trains and evaluates the model
        4. Returns the validation loss for Optuna to minimize
        
        Parameters:
        - trial: Optuna trial object that manages hyperparameter suggestions
        
        Returns:
        - Validation loss (to be minimized)
        """
        # Suggest hyperparameters within defined ranges
        first_lstm_layer_size = trial.suggest_int('first_lstm_layer_size', 32, 256, step=32)
        second_lstm_layer_size = trial.suggest_int('second_lstm_layer_size', 32, 128, step=32)
        dense_layer_size = trial.suggest_int('dense_layer_size', 16, 128, step=16)
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

        # Define the model with suggested hyperparameters
        model = regression_model_builder(
            input_shape=renewables_df_train_scaled.shape,
            first_lstm_layer_size=first_lstm_layer_size,
            second_lstm_layer_size=second_lstm_layer_size,
            dense_layer_size=dense_layer_size,
            kernel_initializer=KERNEL_INITIALIZER,
            recurrent_initializer=RECURRENT_INITIALIZER
        )

        # Compile the model
        # log_cosh loss is used as it's robust for regression problems
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
                      loss='log_cosh',   # Log-cosh loss: smooth approximation of absolute error, less sensitive to outliers
                      metrics=['mse', 'mae'])

        # Train the model with early stopping to prevent overfitting
        model.fit(renewables_X_train, renewables_y_train, 
                  epochs=200,
                  batch_size=64,
                  validation_data=(renewables_X_val, renewables_y_val),
                  callbacks=[generate_early_stop()],
                  verbose=0)

        # Evaluate the model on validation data and return the loss
        return model.evaluate(renewables_X_val, renewables_y_val, verbose=0)[0]

    # Create an Optuna study for hyperparameter optimization
    study: optuna.study.Study = optuna.create_study(direction='minimize', study_name="renewables_model")
    study.optimize(objective, n_trials=200, show_progress_bar=True)

    print("Best hyperparameters:", study.best_params)

    # Train the final model with the best hyperparameters
    renewables_best_params: Dict[str, Any] = study.best_params
    best_model: keras.Model = regression_model_builder(
        input_shape=renewables_df_train_scaled.shape,
        first_lstm_layer_size=renewables_best_params["first_lstm_layer_size"],
        second_lstm_layer_size=renewables_best_params["second_lstm_layer_size"],
        dense_layer_size=renewables_best_params["dense_layer_size"],
        kernel_initializer=KERNEL_INITIALIZER,
        recurrent_initializer=RECURRENT_INITIALIZER
    )

    # Compile with the best learning rate
    best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=renewables_best_params['learning_rate']), 
                       loss='log_cosh', metrics=['mse', 'mae'])
    
    # Train the final model, saving the best weights
    best_model.fit(renewables_X_train[:, :, 0], renewables_y_train,
                   epochs=200, 
                   batch_size=64, 
                   validation_data=(renewables_X_val, renewables_y_val),
                   callbacks=[RENEWABLES_CHECKPOINT, generate_early_stop()])

    # Make predictions
    y_pred_best: np.ndarray = best_model.predict(renewables_X_test)

    y_pred_best_reshaped = y_pred_best.flatten().reshape(-1, 1)
    y_test_reshaped = renewables_y_test.flatten().reshape(-1, 1)

    y_pred_best_original = renewables_scaler.inverse_transform(y_pred_best_reshaped)
    y_test_original = renewables_scaler.inverse_transform(renewables_y_test)

    # Calculate performance metrics
    mape_best = np.mean(np.abs((y_test_original - y_pred_best_original) / y_test_original)) * 100
    r2_best = r2_score(y_test_original, y_pred_best_original)

    # Print performance metrics
    print(f"Best model results:")
    print(f"Mean Absolute Percentage Error (MAPE): {mape_best:.3f}%")
    print(f"R-squared (R2): {r2_best}")

    # Save the best hyperparameters to a JSON file for later use
    hyperparameters_filename = f"best_hyperparameters_renewable.json"
    with open(hyperparameters_filename, 'w') as json_file:
        json.dump(renewables_best_params, json_file, indent=4)

    print(f"Best hyperparameters for 'renewable energy production' saved to {hyperparameters_filename}")

else:
    # If not fine-tuning, load the previously saved best hyperparameters
    hyperparameters_filename = "best_hyperparameters_renewable.json"
    with open(hyperparameters_filename, 'r') as json_file:
        renewables_best_params: Dict[str, Any] = json.load(json_file)

### Non-renewables

In [ ]:
if IS_TO_FINE_TUNE:
    print(f"Fine tunning 'non-renewable energy production' time series")

    def objective(trial: optuna.trial.Trial):
        first_lstm_layer_size = trial.suggest_int('first_lstm_layer_size', 32, 256, step=32)
        second_lstm_layer_size = trial.suggest_int('second_lstm_layer_size', 32, 128, step=32)
        dense_layer_size = trial.suggest_int('dense_layer_size', 16, 128, step=16)
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)

        # Create model with the suggested hyperparameters
        model = regression_model_builder(
            input_shape=non_renewables_df_train_scaled.shape,
            first_lstm_layer_size=first_lstm_layer_size,
            second_lstm_layer_size=second_lstm_layer_size,
            dense_layer_size=dense_layer_size,
            kernel_initializer=KERNEL_INITIALIZER,
            recurrent_initializer=RECURRENT_INITIALIZER
        )

        # Compile model with Adam optimizer and log-cosh loss function
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), loss='log_cosh', metrics=['mse', 'mae'])

        # Train the model
        model.fit(non_renewables_X_train, non_renewables_y_train, epochs=200, batch_size=64, validation_data=(non_renewables_X_val, non_renewables_y_val), callbacks=[generate_early_stop()], verbose=0)

        # Return validation loss
        return model.evaluate(non_renewables_X_val, non_renewables_y_val, verbose=0)[0]

    # Create and run the Optuna study with 200 trials
    study: optuna.study.Study = optuna.create_study(direction='minimize', study_name="non_renewables_model")
    study.optimize(objective, n_trials=200, show_progress_bar=True)

    print("Best hyperparameters:", study.best_params)

    # Train a final model with the best hyperparameters
    non_renewables_best_params: Dict[str, Any] = study.best_params
    best_model: keras.Model = regression_model_builder(
        input_shape=non_renewables_df_train_scaled.shape,
        first_lstm_layer_size=non_renewables_best_params["first_lstm_layer_size"],
        second_lstm_layer_size=non_renewables_best_params["second_lstm_layer_size"],
        dense_layer_size=non_renewables_best_params["dense_layer_size"],
        kernel_initializer=KERNEL_INITIALIZER,
        recurrent_initializer=RECURRENT_INITIALIZER
    )

    # Compile and train the best model
    # Uses checkpointing to save the best model weights during training
    best_model.compile(optimizer=keras.optimizers.Adam(learning_rate=non_renewables_best_params['learning_rate']), loss='log_cosh', metrics=['mse', 'mae'])
    best_model.fit(non_renewables_X_train[:, :, 0], non_renewables_y_train, epochs=200, batch_size=64, validation_data=(non_renewables_X_val, non_renewables_y_val), callbacks=[NON_RENEWABLES_CHECKPOINT, generate_early_stop()])

    # Generate predictions
    y_pred_best: np.ndarray = best_model.predict(non_renewables_X_test)

    # Reshape predictions and true values for inverse scaling
    y_pred_best_reshaped = y_pred_best.flatten().reshape(-1, 1)
    y_test_reshaped = non_renewables_y_test.flatten().reshape(-1, 1)

    # Convert scaled predictions back to original scale for meaningful evaluation
    y_pred_best_original = non_renewables_scaler.inverse_transform(y_pred_best_reshaped)
    y_test_original = non_renewables_scaler.inverse_transform(non_renewables_y_test)

    # Calculate performance metrics
    mape_best = np.mean(np.abs((y_test_original - y_pred_best_original) / y_test_original)) * 100
    r2_best = r2_score(y_test_original, y_pred_best_original)

    # Display final performance metrics
    print(f"Best model results:")
    print(f"Mean Absolute Percentage Error (MAPE): {mape_best:.3f}%")
    print(f"R-squared (R2): {r2_best}")

    # Save the best hyperparameters to JSON for future use
    # This enables model reproducibility without rerunning optimization
    hyperparameters_filename = f"best_hyperparameters_non_renewable.json"
    with open(hyperparameters_filename, 'w') as json_file:
        json.dump(non_renewables_best_params, json_file, indent=4)

    print(f"Best hyperparameters for 'non-renewable energy production' saved to {hyperparameters_filename}")

    # Clean up the Optuna study to free resources
    optuna.delete_study(study_name="non_renewables_model")

else:
    # If not fine-tuning, load previously saved best hyperparameters
    hyperparameters_filename = "best_hyperparameters_non_renewable.json"
    with open(hyperparameters_filename, 'r') as json_file:
        non_renewables_best_params: Dict[str, Any] = json.load(json_file)

## Model Benchmarking and Performance Analysis

### Benchmark Model Fit

After finding the optimal hyperparameters, we need to assess the stability and reliability of our models. Deep learning models can show performance variability due to random initialization, training data batching, and optimization dynamics.

This section implements a comprehensive benchmarking approach by:
1. Training both renewable and non-renewable energy models multiple times (60 iterations)
2. Measuring performance metrics (MAPE, R²) and training duration for each run
3. Storing results for statistical analysis

This benchmarking allows us to establish confidence intervals for expected model performance rather than relying on a single training run.

In [28]:
if IS_TO_BENCHMARK:
    # Define columns for storing benchmark results
    columns = ['attempt', 'mape', 'r2', 'training duration']
    renewables_model_performance = []
    non_renewables_model_performance = []

    # Number of training iterations for reliable statistics
    n_iterations = 60

    # Benchmark the renewables energy forecasting model
    print("Renewables time series")
    for i in range(n_iterations):
        print(f"Attempt {i + 1}..")

        # Create model with best hyperparameters found during optimization
        model: keras.Model = regression_model_builder(
            input_shape=renewables_df_train_scaled.shape,
            first_lstm_layer_size=renewables_best_params["first_lstm_layer_size"],
            second_lstm_layer_size=renewables_best_params["second_lstm_layer_size"],
            dense_layer_size=renewables_best_params["dense_layer_size"]
        )

        # Measure training time for performance benchmarking
        start_ts: float = time.time()
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=renewables_best_params['learning_rate']), loss='log_cosh', metrics=['mse', 'mae'])
        model.fit(renewables_X_train[:, :, 0], renewables_y_train, epochs=200, batch_size=64, validation_data=(renewables_X_val, renewables_y_val), callbacks=[RENEWABLES_CHECKPOINT, generate_early_stop()], verbose=0)
        training_duration: float = time.time() - start_ts   # duration in seconds

        # Generate predictions and evaluate model performance
        y_pred_best: np.ndarray = model.predict(renewables_X_test)

        # Reshape for inverse transformation
        y_pred_reshaped = y_pred_best.flatten().reshape(-1, 1)
        y_test_reshaped = renewables_y_test.flatten().reshape(-1, 1)

        # Convert back to original scale
        y_pred_original = renewables_scaler.inverse_transform(y_pred_reshaped)
        y_test_original = renewables_scaler.inverse_transform(renewables_y_test)

        # Calculate performance metrics
        mape = np.mean(np.abs((y_test_original - y_pred_original) / y_test_original)) * 100
        r2 = r2_score(y_test_original, y_pred_original)

        # Store results for this iteration
        renewables_model_performance.append([i + 1, mape, r2, training_duration])

    # Benchmark the non-renewables energy forecasting model
    print("Non-renewables time series")
    for i in range(n_iterations):
        print(f"Attempt {i + 1}..")
        
        # Create model with best hyperparameters
        model: keras.Model = regression_model_builder(
            input_shape=non_renewables_df_train_scaled.shape,
            first_lstm_layer_size=non_renewables_best_params["first_lstm_layer_size"],
            second_lstm_layer_size=non_renewables_best_params["second_lstm_layer_size"],
            dense_layer_size=non_renewables_best_params["dense_layer_size"]
        )

        # Measure training time
        start_ts: float = time.time()
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=non_renewables_best_params['learning_rate']), loss='log_cosh', metrics=['mse', 'mae'])
        model.fit(non_renewables_X_train[:, :, 0], non_renewables_y_train, epochs=200, batch_size=64, validation_data=(non_renewables_X_val, non_renewables_y_val), callbacks=[RENEWABLES_CHECKPOINT, generate_early_stop()], verbose=0)
        training_duration: float = time.time() - start_ts
        
        # Generate predictions and evaluate model performance
        y_pred_best: np.ndarray = model.predict(non_renewables_X_test)

        # Reshape and convert back to original scale
        y_pred_reshaped = y_pred_best.flatten().reshape(-1, 1)
        y_test_reshaped = non_renewables_y_test.flatten().reshape(-1, 1)
        y_pred_original = non_renewables_scaler.inverse_transform(y_pred_reshaped)
        y_test_original = non_renewables_scaler.inverse_transform(non_renewables_y_test)

        # Calculate performance metrics
        mape = np.mean(np.abs((y_test_original - y_pred_original) / y_test_original)) * 100
        r2 = r2_score(y_test_original, y_pred_original)

        # Store results for this iteration
        non_renewables_model_performance.append([i + 1, mape, r2, training_duration])

    # Convert results to DataFrames and save to CSV for later analysis
    renewables_model_performance_df = pd.DataFrame(renewables_model_performance, columns=columns)
    non_renewables_model_performance_df = pd.DataFrame(non_renewables_model_performance, columns=columns)

    renewables_model_performance_df.to_csv("renewables_model_performance.csv", index=False)
    non_renewables_model_performance_df.to_csv("non_renewables_model_performance.csv", index=False)

else:
    # If not benchmarking, load previous benchmark results
    renewables_model_performance_df = pd.read_csv("renewables_model_performance.csv")
    non_renewables_model_performance_df = pd.read_csv("non_renewables_model_performance.csv")

### Outlier Removal for Robust Performance Estimation

Deep learning models performance can sometimes be affected by good or bad initializations. To get a more robust estimate of expected performance, we apply statistical trimming to our benchmark results.

In this section, we:
1. Sort all benchmark results by MAPE (our primary performance metric)
2. Remove the top and bottom 5% of results (potential outliers)
3. Focus on the middle 90% to establish realistic performance expectations

This approach provides a more reliable estimate of the model's performance in production settings.

In [29]:
# Sort results by MAPE (lower is better) to identify outliers
renewables_model_performance_df.sort_values(by='mape', inplace=True)
non_renewables_model_performance_df.sort_values(by='mape', inplace=True)

# Exclude the best 5% and worst 5% of results to remove potential outliers
n = len(renewables_model_performance_df)
lower_bound = int(n * 0.05)
upper_bound = int(n * 0.95)
renewables_model_performance_df = renewables_model_performance_df.iloc[lower_bound:upper_bound]
non_renewables_model_performance_df = non_renewables_model_performance_df.iloc[lower_bound:upper_bound]

# Reset indices after filtering
renewables_model_performance_df.reset_index(drop=True, inplace=True)
non_renewables_model_performance_df.reset_index(drop=True, inplace=True)

### Performance Metrics Summary

After running multiple benchmark iterations, we can summarize the expected performance range for both our renewable and non-renewable energy forecasting models.

The key metrics we analyze are:
1. **MAPE (Mean Absolute Percentage Error)** - Lower values indicate better forecasting accuracy
2. **R² (Coefficient of Determination)** - Higher values (closer to 100%) indicate the model explains more variance in the data
3. **Training Duration** - Indicates computational requirements and training efficiency

By providing ranges rather than single values, we offer a more realistic picture of expected model performance in production.

In [30]:
# Calculate range of performance metrics for renewable energy model
renewables_mape_range = f"[{round(renewables_model_performance_df["mape"].min(), 3)}%, {round(renewables_model_performance_df["mape"].max(), 3)}%]"
renewables_r2_range = f"[{round(renewables_model_performance_df["r2"].min()  * 100, 3)}%, {round(renewables_model_performance_df["r2"].max()  * 100, 3)}%]"
renewables_duration_range = f"[{round(renewables_model_performance_df["training duration"].min(), 3)}s, {round(renewables_model_performance_df["training duration"].max(), 3)}s]"

# Calculate range of performance metrics for non-renewable energy model
non_renewables_mape_range = f"[{round(non_renewables_model_performance_df["mape"].min(), 3)}%, {round(non_renewables_model_performance_df["mape"].max(), 3)}%]"
non_renewables_r2_range = f"[{round(non_renewables_model_performance_df["r2"].min()  * 100, 3)}%, {round(non_renewables_model_performance_df["r2"].max()  * 100, 3)}%]"
non_renewables_duration_range = f"[{round(non_renewables_model_performance_df["training duration"].min(), 3)}s, {round(non_renewables_model_performance_df["training duration"].max(), 3)}s]"

# Display performance ranges for renewable energy models
print("Renewables model performance (with 90% of confidence):")
print(renewables_mape_range)
print(renewables_r2_range)
print(renewables_duration_range, "\n")

# Display performance ranges for non-renewable energy models
print("Non-renewables model performance (with 90% of confidence):")
print(non_renewables_mape_range)
print(non_renewables_r2_range)
print(non_renewables_duration_range)

Renewables model performance (with 90% of confidence):
[19.781%, 21.822%]
[74.766%, 76.513%]
[13.13s, 19.286s] 

Non-renewables model performance (with 90% of confidence):
[21.16%, 21.456%]
[52.766%, 55.321%]
[8.397s, 19.274s]
